In [15]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from features import ensembleFeature
import itertools
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold
import pymrmr

In [16]:
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [17]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
test_seq = fa_seq("data/Mouse/mouse_indep.fasta")
trainPos_seq = train_seq[0:int(len(train_seq)/2)]
trainNeg_seq = train_seq[int(len(train_seq)/2):]
testPos_seq = test_seq[0:int(len(test_seq)/2)]
testNeg_seq = test_seq[int(len(test_seq)/2):]
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
testLabel = np.append(np.ones(int(len(test_seq)/2)),np.zeros(int(len(test_seq)-len(test_seq)/2)),axis=0)

# 首先比较两种不同的KNN特征分类效果 （经过比较，最终选择按照值作为判断标准进行划分）

In [18]:
#值
trainData = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
KNN_value = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(KNN_value[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7993819847957707

In [19]:
#绝对位置
trainData = np.array(pd.read_csv("features/KNN_position_train.csv").iloc[:,1:])
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
KNN_position = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(KNN_position[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7971611671168912

In [21]:
result_one = np.vstack( (np.mean(KNN_value,axis=1),np.mean(KNN_position,axis=1)))
pd.DataFrame( np.append( np.array(["KNN_value","KNN_position"]).reshape(-1,1),result_one,axis=1)).to_csv("Result/KNN_value_and_position_comparison.csv",header=["name","Sen","Spe","Acc","Mcc","Auc"])

# 先对PSNP、PSDP、PSTP三者单独跟合并进行尝试

In [42]:
#PSNP
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData = np.vstack((trainPos,trainNeg))
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
PSNP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(PSNP[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7844440024574006

In [44]:
#PSDP
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData = np.vstack((trainPos,trainNeg))
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
PSDP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(PSDP[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7673527673850818

In [47]:
#PSTP
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData = np.vstack((trainPos,trainNeg))
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
PSTP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(PSTP[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7700991585955691

In [33]:
#三者合并
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
trainData = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
PSKP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(PSKP[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7818511582155997

# PSCP特征计算

In [19]:
#先计算各类空格，然后找到最优的特征
All = []
for kk in range(7):
    trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,kk)
    trainData_PSCP_kk = np.vstack((trainPos,trainNeg))
    if kk == 0:
        trainData = trainData_PSCP_kk
    else:
        trainData = np.hstack((trainData,trainData_PSCP_kk))
    #trainData = np.vstack((trainPos,trainNeg))
    print(trainData.shape)
    
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,10)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc

    PSCP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
    if kk == 0:
        All = np.mean(PSCP,axis=1)
    else:
        All = np.vstack((All,np.mean(PSCP,axis=1)))
    print(np.mean(PSCP[4]))

(9126, 40)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8111602671544356
(9126, 79)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8193186476017914
(9126, 117)


...

0.8160119308668543
(9126, 225)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.814461053102651
(9126, 259)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8130348091257084


In [20]:
#保存不同k下的最优值
#pd.DataFrame( np.append( np.array(["0","1","2","3","4","5","6"]).reshape(-1,1), All,axis=1)).to_csv("Result/PSCP_k_stack_value.csv",header=["name","Sen","Spe","Acc","Mcc","Auc"])

In [21]:
#最终选择叠加0、1、2三种情况下的值进行合并
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))

trainData = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))


#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]

    model = SVC(C=1.0,gamma="auto")

    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc

PSCP = np.vstack((Sen,Spe,Acc,Mcc,Auc))
print(np.mean(PSCP[4]))

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8195997129404958


# BPB特征

In [28]:
trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData = np.vstack((trainPos,trainNeg))
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
BPB = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(BPB[4])


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7691185817710608

# KNN，按值来

In [23]:
#值
trainData = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
KNN = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(KNN[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7993819847957707

# 合并所有特征 （PSKP、PSCP、BPB、KNN）

In [63]:
# 值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))



trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB,testData_KNN))

#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
All = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(All[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8122570792740131

In [64]:
Auc_All = []
for kfo in range(0,11):
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,kfo)
    print(kfo)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        print(test_index)
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc

    All = np.vstack((Sen,Spe,Acc,Mcc,Auc))
    print(np.mean(Auc))
    Auc_All.append(Auc)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0
0.8123549794056665
1


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8118775652617704
2


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.8118525470639956
9


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=9 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8122698685316664
10


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8122570792740131


In [67]:
import statistics
for i,k in enumerate(Auc_All):
    print(i)
    print(np.mean(k))
    print(np.max(k)-np.min(k))
    print(statistics.variance(k))

0
0.8123549794056665
0.033796094022358725
0.00016966908255427093
1
0.8118775652617704
0.05132528569132888
0.00021809980597770537
2
0.8121196282698817
0.03924064518166537
0.00016221778210369174
3
0.8119574741347062
0.04485729788145432
0.0002474068311858939
4
0.812159578708872
0.039797687227394074
0.0001675495104787733
5
0.8122062238793253
0.06864748664981812
0.0003828023567334408
6
0.8117089930165087
0.037865910983921225
0.00015178945539260304
7
0.812323907836741
0.07989770775630123
0.0005352118398646404
8
0.8118525470639956
0.04984833370538111
0.00023919939715270859
9
0.8122698685316664
0.05186054671823248
0.0002931823454061548
10
0.8122570792740131
0.0365137160403497
0.00013120361487827903


In [118]:
# kf = KFold(10,True,0)
# for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#     print(test_index)

In [51]:
# result_one = np.vstack( (np.mean(PSKP,axis=1),np.mean(PSCP,axis=1),np.mean(BPB,axis=1),np.mean(KNN,axis=1),np.mean(All,axis=1)))
# pd.DataFrame( np.append( np.array(["PSKP","PSCP","BPB","KNN","All"]).reshape(-1,1),result_one,axis=1)).to_csv("Result/Location_comparison.csv",header=["name","Sen","Spe","Acc","Mcc","Auc"])

# 下一步需要来进行特征提取工作

In [18]:
#首先把所有特征表示出来
#值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))



trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData_origin = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB,testData_KNN))

In [20]:
colu = [f'fea_{i}' for i in range(len(trainData[0]))]

In [78]:
#column = ['Class']
#column.append(colu)
trainData = pd.DataFrame(data = trainData,columns=colu)

In [85]:
# mr_339 = pymrmr.mRMR(trainData,'MIQ',339)
# mr_339

In [86]:
# mr_MID_339 = pymrmr.mRMR(trainData,'MID',339)
# mr_MID_339

In [36]:
#ICA降维  sec_ICA  com_ICA loc_ICA
from sklearn.decomposition import FastICA
ica = FastICA(n_components= 100).fit(trainData_origin)
trainData = ica.transform(trainData_origin)
trainData=pd.DataFrame(trainData)
colu = [f'fea_{i}' for i in range(len(trainData[0]))]
traindata = pd.DataFrame(data = trainData,columns=colu)

In [42]:
traindata = pd.DataFrame(data = trainData,columns=colu)

In [ ]:
mr_MIQ_100 = pymrmr.mRMR(traindata,'MIQ',100)
#mr_MIQ_339
mr_MID_100 = pymrmr.mRMR(traindata,'MID',100)
mr_MID_100

In [47]:
pd.DataFrame(   np.hstack( (np.array(["Loc_MRMR_MID","Loc_MRMR_MIQ"]).reshape(-1,1),np.append( np.array(mr_MID_100).reshape(1,-1),np.array(mr_MIQ_100).reshape(1,-1),axis=0)))).to_csv("Result/Loc_ICA_MRMR.csv")

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


# F-score特征选择

In [48]:
def fscore_core(np,nn,xb,xbp,xbn,xkp,xkn):
    '''
    np: number of positive features
    nn: number of negative features
    xb: list of the average of each feature of the whole instances
    xbp: list of the average of each feature of the positive instances
    xbn: list of the average of each feature of the negative instances
    xkp: list of each feature which is a list of each positive instance
    xkn: list of each feature which is a list of each negatgive instance
    reference: http://link.springer.com/chapter/10.1007/978-3-540-35488-8_13
    '''
    def sigmap (i,np,xbp,xkp):
        return sum([(xkp[i][k]-xbp[i])**2 for k in range(int(np))])

    def sigman (i,nn,xbn,xkn):
        #print sum([(xkn[i][k]-xbn[i])**2 for k in range(nn)])
        return sum([(xkn[i][k]-xbn[i])**2 for k in range(int(nn))])

    n_feature = len(xb)
    fscores = []
    for i in range(n_feature):
        fscore_numerator = (xbp[i]-xb[i])**2 + (xbn[i]-xb[i])**2
        fscore_denominator = (1/float(np-1))*(sigmap(i,np,xbp,xkp))+(1/float(nn-1))*(sigman(i,nn,xbn,xkn))
        fscores.append(fscore_numerator/fscore_denominator)

    return fscores

def fscore(feature,classindex):
    '''
    feature: a matrix whose row indicates instances, col indicates features
    classindex: 1 indicates positive and 0 indicates negative
    '''
    n_instance = len(feature)
    n_feature  = len(feature[0])
    np = sum(classindex)
    nn = n_instance - np
    xkp =[];xkn =[];xbp =[];xbn =[];xb=[]
    for i in range(n_feature):
        xkp_i = [];xkn_i = []
        for k in range(n_instance):
            if classindex[k] == 1:
                xkp_i.append(feature[k][i])
            else:
                xkn_i.append(feature[k][i])
        xkp.append(xkp_i)
        xkn.append(xkn_i)
        sum_xkp_i = sum(xkp_i)
        sum_xkn_i = sum(xkn_i)
        xbp.append(sum_xkp_i/float(np))
        xbn.append(sum_xkn_i/float(nn))
        xb.append((sum_xkp_i+sum_xkn_i)/float(n_instance))
    return fscore_core(np,nn,xb,xbp,xbn,xkp,xkn)

In [49]:
f_score_rank = fscore(trainData,trainLabel)
feature = []
for i in range(len(f_score_rank)):
    if not f_score_rank[i]>0:
        f_score_rank[i] = 0
sorted_id = sorted(range(len(f_score_rank)), key=lambda k: f_score_rank[k], reverse=True)

In [117]:
All = []
feature = []
for i in range(len(sorted_id)):
#for i in range(2):
    feature.append(trainData[:,sorted_id[i]])
    trainData_f = np.array(feature).T
    print(trainData_f.shape)
    
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,10)
    for i,[train_index, test_index] in enumerate(kf.split(trainData_f)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        X_train = trainData_f[train_index]
        X_test = trainData_f[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc
    
    print(np.mean(Auc))
    All.append(np.mean(np.vstack((Sen,Spe,Acc,Mcc,Auc)),axis=1))

In [19]:
pd.DataFrame(np.append(np.array(sorted_id).reshape(-1,1),np.array(All),axis=1)).to_csv("Result/Feature_Selection/Location_ICA_F-Score.csv")
#i=124,即使用125个特征的时候达到最优效果

In [18]:
max_auc = np.max(np.array(All)[:,4])
for i in range(len(All)):
    if All[i][4]==max_auc:
        print(i)

124


# 这一步进行SVM-RFE 特征选择

In [52]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

from sklearn import preprocessing
zscore_scaler=preprocessing.StandardScaler()
x = zscore_scaler.fit_transform(trainData)
model = SVC(C=1.0,gamma="auto",kernel="linear")
model.fit(x, trainLabel)
a = model.coef_

KeyboardInterrupt: 

In [7]:
b = a.reshape(-1)
sorted_id = sorted(range(len(b)), key=lambda k: b[k], reverse=True)
#sorted_id

In [10]:
#SVM-RFE
All = []
feature = []
for i in range(len(sorted_id)):
    feature.append(trainData[:,sorted_id[i]])
    trainData_svm = np.array(feature).T
    print(trainData.shape)
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,10)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc

    All.append(np.mean(np.vstack((Sen,Spe,Acc,Mcc,Auc)),axis=1) )

(9126, 1)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


(9126, 2)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


(9126, 3)


...

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


(9126, 338)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


(9126, 339)


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [11]:
max_auc = np.max(np.array(All)[:,4])
for i in range(len(All)):
    if All[i][4]==max_auc:
        print(i)

98


In [13]:
pd.DataFrame(np.append(np.array(sorted_id).reshape(-1,1),np.array(All),axis=1)).to_csv("Result/Feature_Selection/Location_SVM-RFE.csv")
#i=98,即使用99个特征的时候达到最优效果

# MRMR特征选择，包括MRMRMID和MRMRMIQ

In [53]:
MRMR_MID = pd.read_csv("Result/Loc_ICA_MRMR.csv").iloc[0]
MRMR_MIQ = pd.read_csv("Result/Loc_ICA_MRMR.csv").iloc[1]

In [54]:
list_MID = [int(str(i)[4:]) for i in MRMR_MID[2:] ]
list_MIQ = [int(str(i)[4:]) for i in MRMR_MIQ[2:] ]

In [176]:
#MRMR_MID
from thundersvm import SVC
All = []
feature = []
for i in range(len(list_MIQ)):
    feature.append(trainData[:,list_MIQ[i]])
    trainData_M = np.array(feature).T
    print(trainData_M.shape)
    
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,10)
    for i,[train_index, test_index] in enumerate(kf.split(trainData_M)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        X_train = trainData_M[train_index]
        X_test = trainData_M[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc
    print(np.mean(Auc))

    All.append(np.mean(np.vstack((Sen,Spe,Acc,Mcc,Auc)),axis=1) )

(9126, 1)
0.523156916294414
(9126, 2)
0.523108686762986
(9126, 3)
0.5448848993953148
(9126, 4)
0.5445206157968737
(9126, 5)
0.5544970843206852
(9126, 6)
0.557076120912401
(9126, 7)
0.5581025232356787
(9126, 8)
0.5621483317299862
(9126, 9)
0.5621487565760741
(9126, 10)
0.5642444906966996
(9126, 11)
0.5700706717805322
(9126, 12)
0.5737679750600838
(9126, 13)
0.5779462227759968
(9126, 14)
0.5806201649913777
(9126, 15)
0.5994534237800669
(9126, 16)
0.5995903346924171
(9126, 17)
0.6002766724381405
(9126, 18)
0.6049168862199051
(9126, 19)
0.6068192107671616
(9126, 20)
0.6075141444714676
(9126, 21)
0.609338697224412
(9126, 22)
0.61603233801552
(9126, 23)
0.6210992788605538
(9126, 24)
0.6225699957948331
(9126, 25)
0.6528843855008224
(9126, 26)
0.6526395717826141
(9126, 27)
0.6531914388142421
(9126, 28)
0.6548201480175931
(9126, 29)
0.6583539064652709
(9126, 30)
0.6873819886760337
(9126, 31)
0.6967483351837915
(9126, 32)
0.7026910763702816
(9126, 33)
0.7126697584571404
(9126, 34)
0.712988016393

In [177]:
max_auc = np.max(np.array(All)[:,4])
for i in range(len(All)):
    if All[i][4]==max_auc:
        print(i)

96


In [178]:
pd.DataFrame(np.append(np.array(list_MIQ).reshape(-1,1),np.array(All),axis=1)).to_csv("Result/Feature_Selection/Location_ICA_MRMR_MIQ.csv")
#i=108,即使用109个特征的时候达到最优效果 MID
#i=237,即使用238个特征的时候达到最优效果 MIQ

#i=98,即使用99个特征的时候达到最优效果 MID_ICA
#i=96,即使用97个特征的时候达到最优效果 MIQ

In [186]:
list_MIQ

[69,
 70,
 66,
 67,
 68,
 64,
 62,
 71,
 72,
 73,
 65,
 75,
 63,
 56,
 57,
 53,
 54,
 55,
 76,
 61,
 58,
 59,
 60,
 74,
 51,
 94,
 95,
 91,
 92,
 93,
 89,
 87,
 96,
 97,
 98,
 90,
 50,
 88,
 81,
 82,
 78,
 79,
 80,
 77,
 86,
 83,
 84,
 85,
 52,
 1,
 19,
 20,
 16,
 17,
 18,
 14,
 12,
 21,
 22,
 23,
 15,
 25,
 13,
 6,
 7,
 3,
 4,
 5,
 26,
 11,
 8,
 9,
 10,
 24,
 2,
 44,
 45,
 41,
 42,
 43,
 39,
 37,
 46,
 47,
 48,
 40,
 49,
 38,
 31,
 32,
 28,
 29,
 30,
 27,
 36,
 33,
 34,
 35,
 99,
 0]

# 遗传算法筛选特征

In [120]:
# 值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData_origin = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB,testData_KNN))

from sklearn.decomposition import FastICA
ica = FastICA(n_components= 100).fit(trainData_origin)
trainData = ica.transform(trainData_origin)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [119]:
import numpy as np
from sklearn.model_selection import train_test_split
from thundersvm import SVC

class GA:
    def __init__(self, population_size, p_crossover, p_mutation, n_generations):
        self.population_size = population_size
        self.p_crossover = p_crossover
        self.p_mutation = p_mutation
        self.n_generations = n_generations
        
    def generate_population(self, chromosome_length):
        population = np.random.randint(0, 2, size=(self.population_size, chromosome_length))
        return population
    
    def decode(self, population, feature_list):
        feature_index = []
        for chromosome in population:
            index = [i for (i, v) in zip(range(len(feature_list)), chromosome) if v > 0]
            feature_index.append(index)
        return feature_index

    def fitness(self, X_train, y_train, X_test, y_test, feature_index):
        clf = SVC(kernel='polynomial')
        score_list = []
        for index in feature_index:
            clf.fit(X_train.iloc[:, index], y_train)
            score = clf.score(X_test.iloc[:, index], y_test)
            score_list.append(score)
        return score_list
        
    def selection(self, population, scores):
        new_population = np.zeros_like(population)
        
        for i in range(self.population_size):
            i1, i2 = np.random.choice(range(self.population_size), size=2, replace=False)
            if scores[i1] >= scores[i2]:
                new_population[i] = population[i1]
            else:
                new_population[i] = population[i2]
                
        return new_population
    
    def crossover(self, population):
        new_population = np.zeros((self.population_size, len(population[0])))
        
        for i in range(self.population_size):
            if np.random.rand() < self.p_crossover:
                i1, i2 = np.random.choice(range(self.population_size), size=2, replace=False)
                cross_point = np.random.randint(1, len(population[0]) - 1)
                chrom_1 = np.concatenate((population[i1][:cross_point], population[i2][cross_point:]))
                chrom_2 = np.concatenate((population[i2][:cross_point], population[i1][cross_point:]))
                new_population[i] = np.where(chrom_1 != chrom_2, chrom_1, population[i1])
            else:
                new_population[i] = population[i]
                
        return new_population
    
    def mutation(self, population):
        new_population = np.copy(population)
        
        for i in range(self.population_size):
            for j in range(len(population[0])):
                if np.random.rand() < self.p_mutation:
                    new_population[i][j] = 0 if new_population[i][j] == 1 else 1
                    
        return new_population

    def run(self, X_train, y_train, X_test, y_test, feature_list):
        chromosome_length = len(feature_list)
        population = self.generate_population(chromosome_length)
        
        for i in range(self.n_generations):
            print(i)
            feature_index = self.decode(population, feature_list)
            scores = self.fitness(X_train, y_train, X_test, y_test, feature_index)
            best_score_index = np.argmax(scores)
            best_score = scores[best_score_index]
            
            new_population = self.selection(population, scores)
            new_population = self.crossover(new_population)
            new_population = self.mutation(new_population)
            
            new_feature_index = self.decode(new_population, feature_list)
            new_scores = self.fitness(X_train, y_train, X_test, y_test, new_feature_index)
            new_best_score_index = np.argmax(new_scores)
            new_best_score = new_scores[new_best_score_index]
            
            if new_best_score > best_score:
                population = new_population
            else:
                population[best_score_index] = new_population[new_best_score_index]
                
        best_chromosome = population[np.argmax(self.fitness(X_train, y_train, X_test, y_test, self.decode(population, feature_list)))]
        return best_chromosome

In [168]:
# import pandas as pd
# #from GA import GA
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split

# kf = KFold(10,True,0)
# for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#     if i==0:
#         X_train = trainData[train_index]
#         X_test = trainData[test_index]
#         Y_train = trainLabel[train_index]
#         Y_test = trainLabel[test_index]
    
# # # 数据准备
# # X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)

# # # 将数据分割为训练集和测试集
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 将训练集和测试集转换成 DataFrame 格式，并设置列名
# X_train = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])
# X_test = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X_test.shape[1])])
# Y_train = pd.Series(Y_train)
# Y_test = pd.Series(Y_test)

# # 初始化 GA 类
# population_size = 50
# p_crossover = 0.8
# p_mutation = 0.2
# n_generations = 500

# ga = GA(population_size, p_crossover, p_mutation, n_generations)

# # 运行遗传算法并返回选择的变量序列
# selected_features = ga.run(X_train, Y_train, X_test, Y_test, list(X_train.columns))

In [125]:
sum(selected_features)

44

In [126]:
def select_features(data,features_list):
    selected_features = []
    
    # 遍历列表中的每个元素，如果该元素的值为1，则将该元素的索引添加到selected_features列表中
    for i in range(len(features_list)):
        if features_list[i] == 1:
            selected_features.append(data[:,i])
            
    return selected_features

a = select_features(trainData,selected_features)
a = np.array(a).T

In [127]:
Sen = []
Spe = []
Acc = []
Mcc = []
Auc = []
kf = KFold(10,True,0)
for i,[train_index, test_index] in enumerate(kf.split(a)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = a[train_index]
    X_test = a[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]

    model = SVC(C=1.0,gamma="auto")

    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen.append(sen)
    Spe.append(spe)
    Acc.append(acc)
    Mcc.append(mcc)
    Auc.append(auc)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [128]:
np.mean(Sen),np.mean(Spe),np.mean(Acc),np.mean(Mcc),np.mean(Auc)

(0.7015000716035631,
 0.6444135176768863,
 0.6729082598335928,
 0.3467636969594163,
 0.7300854638279904)

In [130]:
num = 44
param_loc = [0.7015000716035631, 0.6444135176768863, 0.6729082598335928, 0.3467636969594163, 0.7300854638279904]

In [131]:
num = 53
param_com = [0.6858035787311453, 0.6987101645766827, 0.6921984589074, 0.38504131407208597, 0.7657598728057325]

In [132]:
num = 48
param_sec = [0.7065719661910479, 0.6947851895710855, 0.7005203829672757, 0.4019167588976491, 0.770064068006594]

# shap-boruta选择  降维后特征

In [136]:
from lightgbm import *
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE
import warnings
warnings.simplefilter('ignore')

In [143]:
df = pd.DataFrame(trainData)

In [144]:
np.random.seed(123)

for c in range(1,101):
    df[f"RANDOM_{c}"] = np.random.normal(0,5, (df.shape[0],1))

In [165]:
### DEFINE TUNING VARIABLES ###

param_grid = {
    'n_estimators': [200,500],
    'learning_rate': [0.3, 0.1],
    'num_leaves': [20, 30]
}


def AUC(y_true, y_hat):
    return 'auc', roc_auc_score(y_true, y_hat), True

lgbm = LGBMClassifier(random_state=0, n_jobs=-1)

n_trials = 100

In [166]:
### TUNE PARAMETERS AND EXTRACT IMPORTANCES ###
from sklearn.metrics import roc_auc_score
importances = np.zeros((df.shape[1],))
df = np.array(df)
for i in range(n_trials):
    print("----- TRIAL {} -----".format(i+1))
    
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(df)):
        X_train = df[train_index]
        X_test = df[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        model = BoostSearch(lgbm, param_grid=param_grid, greater_is_better=True, verbose=0)
        model.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], eval_metric=AUC, early_stopping_rounds=4, verbose=0)
        importances += model.estimator_.feature_importances_
importances /= n_trials
    
    
#     X_train, X_test, y_train, y_test = train_test_split(
#         df, pd.DataFrame(trainLabel), test_size=0.5, shuffle=True, random_state=i)

#     X_train, X_valid, y_train, y_valid = train_test_split(
#         X_train, y_train, test_size=0.2, shuffle=True, random_state=i)

#     model = BoostSearch(lgbm, param_grid=param_grid, greater_is_better=True, verbose=0)
#     model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=AUC, early_stopping_rounds=4, verbose=0)
    
#     importances += model.estimator_.feature_importances_
    
importances /= n_trials

----- TRIAL 1 -----
----- TRIAL 2 -----
----- TRIAL 3 -----
----- TRIAL 4 -----
----- TRIAL 5 -----
----- TRIAL 6 -----
----- TRIAL 7 -----
----- TRIAL 8 -----
----- TRIAL 9 -----
----- TRIAL 10 -----
----- TRIAL 11 -----
----- TRIAL 12 -----
----- TRIAL 13 -----
----- TRIAL 14 -----
----- TRIAL 15 -----
----- TRIAL 16 -----
----- TRIAL 17 -----
----- TRIAL 18 -----
----- TRIAL 19 -----
----- TRIAL 20 -----
----- TRIAL 21 -----
----- TRIAL 22 -----
----- TRIAL 23 -----
----- TRIAL 24 -----
----- TRIAL 25 -----
----- TRIAL 26 -----
----- TRIAL 27 -----
----- TRIAL 28 -----
----- TRIAL 29 -----
----- TRIAL 30 -----
----- TRIAL 31 -----
----- TRIAL 32 -----
----- TRIAL 33 -----
----- TRIAL 34 -----
----- TRIAL 35 -----
----- TRIAL 36 -----
----- TRIAL 37 -----
----- TRIAL 38 -----
----- TRIAL 39 -----
----- TRIAL 40 -----
----- TRIAL 41 -----
----- TRIAL 42 -----
----- TRIAL 43 -----
----- TRIAL 44 -----
----- TRIAL 45 -----
----- TRIAL 46 -----
----- TRIAL 47 -----
----- TRIAL 48 -----
-

In [167]:
data = importances[0:100]

idx = sorted(range(len(data)), key=lambda i: data[i],reverse=True)  # 获取排序后的索引列表

sorted_data = [data[i] for i in idx]  # 使用排序后的索引列表排序原始数据

print(idx)  # 输出排序后的索引列表

[78, 51, 89, 87, 96, 93, 5, 37, 57, 28, 44, 32, 46, 31, 34, 17, 12, 82, 26, 41, 59, 49, 86, 8, 33, 85, 38, 13, 24, 88, 52, 84, 42, 7, 68, 66, 14, 69, 27, 65, 60, 1, 43, 92, 55, 6, 77, 53, 97, 45, 50, 9, 74, 71, 58, 15, 39, 21, 72, 91, 48, 73, 16, 19, 81, 76, 47, 10, 75, 2, 54, 98, 11, 3, 20, 36, 56, 63, 30, 0, 70, 83, 22, 94, 35, 64, 61, 95, 79, 25, 62, 4, 29, 99, 23, 18, 90, 67, 40, 80]


In [169]:
col_boruta_shap = [ "fea_"+str(i) for i in idx]
trainData_loc_ICA = trainData

In [171]:
All = []
feature = []
for index_idx in range(len(idx)):
    feature.append(trainData_loc_ICA[:,index_idx])
    trainData = np.array(feature).T
    print(trainData.shape)
    
    #交叉验证
    rows = 10
    row = -1
    row_name = []
    Sen = np.zeros(rows)
    Spe = np.zeros(rows)
    Acc = np.zeros(rows)
    Mcc = np.zeros(rows)
    Auc = np.zeros(rows)

    kf = KFold(10,True,10)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        row+=1
        row_one = "Bert6mA_origin_KFold_" + str(i)
        row_name.append(row_one)
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]

        model = SVC(C=1.0,gamma="auto")

        model.fit(X_train,Y_train)
        th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
        _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
        Sen[row] = sen
        Spe[row] = spe
        Acc[row] = acc
        Mcc[row] = mcc
        Auc[row] = auc
    print(np.mean(Auc))

    All.append(np.mean(np.vstack((Sen,Spe,Acc,Mcc,Auc)),axis=1) )

(9126, 1)
0.5063387455009846
(9126, 2)
0.5186417939884882
(9126, 3)
0.5309500487029414
(9126, 4)
0.5307664861740011
(9126, 5)
0.5312177529871234
(9126, 6)
0.5574328773333803
(9126, 7)
0.5581197160557068
(9126, 8)
0.5586071560120945
(9126, 9)
0.5665239240449761
(9126, 10)
0.567053452650454
(9126, 11)
0.5711019720972004
(9126, 12)
0.5730317073474467
(9126, 13)
0.5853462084507639
(9126, 14)
0.5899338707533159
(9126, 15)
0.5900795149577942
(9126, 16)
0.5918115503370565
(9126, 17)
0.5913961718550347
(9126, 18)
0.598429570031984
(9126, 19)
0.5983928593769164
(9126, 20)
0.6012278183229791
(9126, 21)
0.6029947672429987
(9126, 22)
0.6058961945353911
(9126, 23)
0.6063971728630944
(9126, 24)
0.6065809575118581
(9126, 25)
0.6080568829613926
(9126, 26)
0.6080734761950184
(9126, 27)
0.6137172648211957
(9126, 28)
0.6173635193254178
(9126, 29)
0.6294532786018281
(9126, 30)
0.6300794303726148
(9126, 31)
0.6298624920574204
(9126, 32)
0.6392358556732175
(9126, 33)
0.6428293679553257
(9126, 34)
0.64544033

In [172]:
max_auc = np.max(np.array(All)[:,4])
for i in range(len(All)):
    if All[i][4]==max_auc:
        print(i)
        print(All[i])

99
[0.77603071 0.7304994  0.75345088 0.50810379 0.83438474]


In [174]:
pd.DataFrame(np.append(np.array(col_boruta_shap).reshape(-1,1),np.array(All),axis=1)).to_csv("Result/Feature_Selection/Location_ICA_boruta_shap.csv",header=["Name","Sen","Spe","Acc","Mcc","Auc"])

# 最终结果--Location_MRMR_MIQ

In [180]:
#遗传算法
num = 44
param_loc = [0.7015000716035631, 0.6444135176768863, 0.6729082598335928, 0.3467636969594163, 0.7300854638279904]

In [181]:
# F-score得分
All = np.array(pd.read_csv("Result/Feature_Selection/Location_F-Score.csv"))
max_auc = max(All[:,-1])
for i in range(len(All)):
    if All[i][-1]==max_auc:
        print(i)
        print(All[i])
        print(All[i][-1])

124
[124.         212.           0.7756708    0.70012122   0.73777683
   0.47713796   0.81371342]
0.8137134207869604


In [182]:
# PCA_boruta-shap得分
All = np.array(pd.read_csv("Result/Feature_Selection/Location_ICA_boruta_shap.csv"))
max_auc = max(All[:,-1])
for i in range(len(All)):
    if All[i][-1]==max_auc:
        print(i)
        print(All[i])
        print(All[i][-1])

99
[99 'fea_80' 0.7760307078772295 0.7304994028783722 0.7534508848792298
 0.508103793684299 0.8343847405117952]
0.8343847405117952


In [183]:
# PCA_MRMR_MID得分
All = np.array(pd.read_csv("Result/Feature_Selection/Location_ICA_MRMR_MID.csv"))
max_auc = max(All[:,-1])
for i in range(len(All)):
    if All[i][-1]==max_auc:
        print(i)
        print(All[i])
        print(All[i][-1])

98
[98.         99.          0.75221434  0.75506714  0.7533416   0.50802249
  0.83445339]
0.8344533882495329


In [184]:
# PCA_MRMR_MIQ得分
All = np.array(pd.read_csv("Result/Feature_Selection/Location_ICA_MRMR_MIQ.csv"))
max_auc = max(All[:,-1])
for i in range(len(All)):
    if All[i][-1]==max_auc:
        print(i)
        print(All[i])
        print(All[i][-1])

96
[96.         34.          0.78666681  0.71845561  0.75268358  0.50665414
  0.83448953]
0.8344895294777729


In [185]:
# PCA_SVM_RFE得分
All = np.array(pd.read_csv("Result/Feature_Selection/Location_SVM-RFE.csv"))
max_auc = max(All[:,-1])
for i in range(len(All)):
    if All[i][-1]==max_auc:
        print(i)
        print(All[i])
        print(All[i][-1])

98
[ 98.         125.           0.76824721   0.71965997   0.74391537
   0.48835974   0.81607763]
0.816077634352568
